In [1]:
from pathlib import Path
import sys
import pandas as pd

ROOT = Path.cwd().parents[0] if Path.cwd().name == "notebooks" else Path.cwd()
sys.path.append(str(ROOT))

print("ROOT:", ROOT)
print("sys.path contains ROOT:", str(ROOT) in sys.path)

from src.modeling import (
    train_val_test_split,
    scale_numeric,
    train_logistic_regression,
    train_random_forest,
    train_gradient_boosting,
    evaluate_classifier,
)
from src.visualization import plot_confusion_matrix, plot_roc_curve

df_loans_fe = pd.read_csv(ROOT / "data/processed/loan_default_features.csv")
df_credit_fe = pd.read_csv(ROOT / "data/processed/credit_risk_features.csv")

ROOT: /Users/erincheng/Desktop/credit-risk-analysis
sys.path contains ROOT: True


In [2]:
target = "default"
df = df_loans_fe.copy()

numeric_cols = [
    "age", "income", "loan_amount", "credit_score", "months_employed", "num_credit_lines", "interest_rate", "loan_term", "dti_ratio",
    ]

x_train, x_val, x_test, y_train, y_val, y_test = train_val_test_split(
    df, target =target
)

x_train_scaled, x_val_scaled, x_test_scaled, scaler = scale_numeric(
    x_train, x_val, x_test, numeric_cols
)

In [3]:
# logistic regression
log_reg = train_logistic_regression(x_train_scaled, y_train)

# random forest
rf = train_random_forest(x_train, y_train)

# gradient boosting
gb = train_gradient_boosting(x_train, y_train)

InvalidParameterError: The 'solver' parameter of LogisticRegression must be a str among {'liblinear', 'saga', 'sag', 'lbfgs', 'newton-cg', 'newton-cholesky'}. Got '1bfgs' instead.

In [3]:
models = {
    "log_reg": (log_reg, x_val_scaled, x_test_scaled),
    "random_forest": (rf, x_val, x_test),
    "gradient_boosting": (gb, x_val, x_test),
}

results = {}

for name, (model, xv, xt) in models.items():
    val_metrics = evaluate_classifier(model, xv, y_val)
    test_metrics = evaluate_classifier(model, xt, y_test)
    results[name] = {"val": val_metrics, "test": test_metrics}
    
results

NameError: name 'log_reg' is not defined

In [ ]:
best_model = rf

# confusion matric and ROC on test set
plot_confusion_matrix(best_model, x_test, y_test, title="Loan Default - Random Forest")
plot_roc_curve(best_model, x_test, y_test, title="Loan Default - Random Forest ROC")